In [18]:
import nltk
import string
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import snowball
from nltk.corpus import stopwords
import keras
from keras import *
from keras import layers
from keras.preprocessing import text
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import PyPDF2
from glob import glob, glob1
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\doonl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\doonl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
path = './txtFile/'

data = pd.DataFrame(columns=['id', 'abstract', 'keywords'])
for i in glob1('./txtFile/', '*'):
    with open(path + i, 'r', encoding='utf8') as f:
        doc_id = i.split('.')[-2]
        readingAbstract = False
        readingKeywords = False
        abstract_sent = []
        keyword_list = []
        keyword_clean = []
        for j in f:
            line = j.strip()
            if line == 'Keywords:' and readingKeywords is False:
                readingKeywords = True
                readingAbstract = False
                continue
            if line == 'Abstract' and readingAbstract is False:
                readingAbstract = True
                continue
            if readingAbstract is True:
                abstract_sent.append(line)
            if readingKeywords is True:
                keyword_list.append(line)
        abstract_para = ' '.join(abstract_sent)
        keyword_split = keyword_list[0].split(',')
        # print(abstract_para)
        # print(keyword_split)
        for x in keyword_split:
            keyword_lower = x.lower()
            keyword_no_space = keyword_lower.replace(' ', '')
            keyword_underscore = keyword_no_space.replace('_', ' ')
            keyword_clean.append(keyword_underscore)
        # print(keyword_clean)

        data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
    f.close()

display(data)


C:\Users\doonl\AppData\Local\Temp\ipykernel_22476\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_22476\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_22476\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_22476\2526147678.py:36:

,id,abstract,keywords
0,1,"In the logistic management, the cost reduction...","[artificial intelligence, genetic algorithm]"
1,10,Word segmentation is an important pre-processi...,[deep learning]
2,11,Loss estimation is considerably significant fo...,[network resource planning]
3,12,This thesis presents a computational study on...,"[audio-synchronization, facial expression]"
4,13,The financial problem is a challenging task fo...,"[deep learning, distributional reinforcement l..."
5,14,Since a classical computer has a limitation to...,"[quantum random walk, quantum computer simulator]"
6,15,Hyperparameter Tuning chooses optimal hyperpar...,"[hyperparameter tuning, random search]"
7,16,The honeybee is a social insect that communica...,"[convolutional neural networks, feature extrac..."
8,17,System logs contain the complete information o...,"[alice, convolutional neural network]"
9,18,Handling an imbalanced class problem is a chal...,"[classification, probabilistic distribution]"


In [54]:
data.count()

id          18
abstract    18
keywords    18
dtype: int64

In [55]:
data_explode = data.explode('keywords').reset_index(drop=True)

In [56]:
data_explode

,id,abstract,keywords
0,1,"In the logistic management, the cost reduction...",artificial intelligence
1,1,"In the logistic management, the cost reduction...",genetic algorithm
2,10,Word segmentation is an important pre-processi...,deep learning
3,11,Loss estimation is considerably significant fo...,network resource planning
4,12,This thesis presents a computational study on...,audio-synchronization
5,12,This thesis presents a computational study on...,facial expression
6,13,The financial problem is a challenging task fo...,deep learning
7,13,The financial problem is a challenging task fo...,distributional reinforcement learning
8,14,Since a classical computer has a limitation to...,quantum random walk
9,14,Since a classical computer has a limitation to...,quantum computer simulator


In [57]:
data_u = data_explode['keywords'].unique()
data_u

array(['artificial intelligence', 'genetic algorithm', 'deep learning',
       'network resource planning', 'audio-synchronization',
       'facial expression', 'distributional reinforcement learning',
       'quantum random walk', 'quantum computer simulator',
       'hyperparameter tuning', 'random search',
       'convolutional neural networks', 'feature extraction', 'alice',
       'convolutional neural network', 'classification',
       'probabilistic distribution', 'gradient boosting', 'cern',
       'long short term memory', 'abstractive text summarization',
       'covid-19', 'acoustic-to-articulatory inversion',
       'natural language processing', 'topic modeling', 'document mining',
       'graph analysis'], dtype=object)

In [58]:
data_explode['keywords'].duplicated().any

<bound method NDFrame._add_numeric_operations.<locals>.any of 0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18     True
19    False
20     True
21    False
22     True
23    False
24    False
25    False
26    False
27     True
28    False
29    False
30    False
31    False
32     True
Name: keywords, dtype: bool>

In [59]:
data_explode['word_id'] = pd.factorize(data_explode.keywords)[0]
data_explode

,id,abstract,keywords,word_id
0,1,"In the logistic management, the cost reduction...",artificial intelligence,0
1,1,"In the logistic management, the cost reduction...",genetic algorithm,1
2,10,Word segmentation is an important pre-processi...,deep learning,2
3,11,Loss estimation is considerably significant fo...,network resource planning,3
4,12,This thesis presents a computational study on...,audio-synchronization,4
5,12,This thesis presents a computational study on...,facial expression,5
6,13,The financial problem is a challenging task fo...,deep learning,2
7,13,The financial problem is a challenging task fo...,distributional reinforcement learning,6
8,14,Since a classical computer has a limitation to...,quantum random walk,7
9,14,Since a classical computer has a limitation to...,quantum computer simulator,8


In [60]:
data_explode[data_explode['keywords'] == 'deep learning'].tail()

,id,abstract,keywords,word_id
2,10,Word segmentation is an important pre-processi...,deep learning,2
6,13,The financial problem is a challenging task fo...,deep learning,2
22,4,Due to the emerging of Long Short-Term Memory ...,deep learning,2
27,6,This thesis proposed a speech production acqui...,deep learning,2


In [61]:
data_explode.describe()

,word_id
count,33.000000
mean,11.939394
std,8.306396
min,0.000000
25%,4.000000
50%,12.000000
75%,19.000000
max,26.000000


In [62]:
x = np.array(data_explode['abstract'].values)
y = np.array(data_explode['word_id'].values)

In [63]:
len(y)

33

In [64]:
x

array(['In the logistic management, the cost reduction for delivering the  goods to customer s is crucial to save the budget of the company . Since decades ago, various Vehicle Routing Problems (VRPs ) have been emerged enormously to improve the productivity and to reduce the logistic cost of the industry . Among them, Vehicle Routing Problem with Time Windows  (VRPTW ) is one of the most fundamental VRP variants and one of the most applicable variants in the real-world case studies. In this study , VRPTW with  hard time windows is solved  by developing a special Genetic Algorithm (GA), composed of a problem -specific crossover operator and seven different mutation operators. The proposed GA has better results with  the heuristic mutation among seven operators while exploring the new and better features in large search space. The results of the algorithm are tested on the popular Solomon benchmark 100 - customer datasets. The results show that the proposed GA is quite comparable  with 

In [65]:
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.1, random_state=0)


In [66]:
len(x_train[0])
x_train

array(['Word segmentation is an important pre-processing step in natural language processing applications, particularly in languages with no demarcation indicators including such as Thai. Dictionary-based segmentation, while simple, does not consider the context of the sentence. This paper proposes an attention-based deep learning approach for Thai word segmentation. With an additional attention mechanism, the model can learn character correlations across the entire sentence without gradient vanishing or gradient explode problems and then tokenize them into word vectors. The goal of this research is to test two different types of attention mechanisms to determine the effectiveness of word tokenization. The visualization of attention for each attention mechanism is also included as an outcome.',
       'Since the spread of Coronavirus disease or Covid-19 at the end of 2019, there has been an extensive amount of news about Covid-19, and it takes a long time for humans to read the news, p

In [67]:
t = text.Tokenizer(filters = '!"#$%&()*+-./:;<=>?@[\]^_`{|}~', lower = True)
t.fit_on_texts(x_train)
x_indx = t.texts_to_sequences(x_train)
x_indx_test = t.texts_to_sequences(x_test)

In [68]:
len(x_indx)
len(x_indx_test)

4

In [69]:
maxlen = len(max(x_indx, key=len))


In [70]:
len(t.word_index)

1205

In [127]:
for i, x in enumerate(x_indx_test):
    if len(x) < maxlen:
        x_indx_test[i] = np.hstack((x, np.zeros(maxlen-len(x))))
x_test_index = np.array(x_indx_test)


In [128]:
x_test_index.shape

(4, 278)

In [71]:
for i, x in enumerate(x_indx):
    if len(x) < maxlen:
        x_indx[i] = np.hstack((x, np.zeros(maxlen-len(x))))
x_train_index = np.array(x_indx)

In [72]:
x_train_index.shape

(29, 278)

In [75]:
x_train_index[28]

array([  5.,   1., 335., 790.,   1., 191., 270.,  12., 791.,   1., 792.,
         4., 171.,  70.,   7., 793.,   4., 794.,   1., 795.,   2.,   1.,
       796., 202., 797., 798., 149., 336., 337.,  92., 799.,  55.,  57.,
       800., 801.,   4., 177.,   1., 802.,   3.,   4., 392.,   1., 335.,
       191.,   2.,   1., 803., 338., 804., 336., 337.,  79.,   9.,  40.,
       339., 340.,   7.,  67.,   2.,   1., 109., 334., 341., 342.,   3.,
        67.,   2.,   1., 109., 805., 342.,   5.,   1., 120., 148., 372.,
       316.,   5.,  10.,  30.,  13., 340.,   9., 427.,  40., 339.,   7.,
       806.,  28., 428.,   6., 807., 135.,  48.,  64.,  13., 808.,   2.,
         6.,  79., 208., 809., 810.,   3., 175.,  51., 343., 186.,   1.,
        29.,  64.,  52.,  77.,  32.,   9.,   1., 811., 343., 338., 175.,
       186., 140., 812.,   1.,  47.,   3.,  77.,  50.,   5., 122., 152.,
       813.,   1.,  32.,   2.,   1.,  48.,  25., 320.,   8.,   1., 178.,
       344., 216., 814., 171., 107.,   1.,  32., 21

In [76]:
total_word = len(t.word_index) +1

In [77]:
t.word_index

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'in': 5,
 'a': 6,
 'is': 7,
 'on': 8,
 'with': 9,
 'this': 10,
 'from': 11,
 'for': 12,
 ',': 13,
 'model': 14,
 'data': 15,
 'learning': 16,
 'using': 17,
 'that': 18,
 'an': 19,
 'based': 20,
 'were': 21,
 'network': 22,
 'was': 23,
 'system': 24,
 'are': 25,
 'performance': 26,
 'word': 27,
 'by': 28,
 'proposed': 29,
 'study': 30,
 'thai': 31,
 'results': 32,
 'quantum': 33,
 'can': 34,
 'applied': 35,
 'dataset': 36,
 'which': 37,
 'as': 38,
 'lstm': 39,
 'time': 40,
 'be': 41,
 'used': 42,
 'than': 43,
 'many': 44,
 'it': 45,
 'speech': 46,
 'new': 47,
 'algorithm': 48,
 'analysis': 49,
 'features': 50,
 'different': 51,
 'has': 52,
 'method': 53,
 'we': 54,
 'have': 55,
 'language': 56,
 'been': 57,
 'process': 58,
 'models': 59,
 'other': 60,
 'our': 61,
 'lda': 62,
 'circuit': 63,
 'ga': 64,
 'text': 65,
 'neural': 66,
 'one': 67,
 'machine': 68,
 'random': 69,
 's': 70,
 'number': 71,
 'best': 72,
 'processing': 73,
 'research': 74,


In [78]:
total_word

1206

In [79]:
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity


In [119]:
model = keras.Sequential()

model.add(keras.layers.Embedding(total_word, 100, input_length=maxlen, name='Embedding'))
model.add(keras.layers.LSTM(128,activation = 'tanh', return_sequences=True))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dense(27, activation='softmax'))
# model.add(keras.layers.Dense(70))

model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop' ,metrics=['acc'])



In [120]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 278, 100)          120600    
                                                                 
 lstm_16 (LSTM)              (None, 278, 128)          117248    
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dense_8 (Dense)             (None, 27)                3483      
                                                                 
Total params: 372,915
Trainable params: 372,915
Non-trainable params: 0
_________________________________________________________________


In [113]:
y_train = np.array(y_train)

In [114]:
len(y_train)

29

In [125]:
his = model.fit(x_train_index, y_train, epochs=20, batch_size= 32)

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 1.9741 - acc: 0.3103
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 1.9550 - acc: 0.3103
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 2.0471 - acc: 0.2759
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 2.5198 - acc: 0.2069
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 2.4275 - acc: 0.2069
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 2.1925 - acc: 0.2759
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 2.1042 - acc: 0.3103
Epoch 8/20
1/1 [==============================] - 3s 3s/step - loss: 2.1076 - acc: 0.3103
Epoch 9/20
1/1 [==============================] - 3s 3s/step - loss: 2.3730 - acc: 0.2069
Epoch 10/20
1/1 [==============================] - 3s 3s/step - loss: 2.0033 - acc: 0.2759
Epoch 11/20
1/1 [==============================] - 3s 3s/step - loss: 2.0381 - acc: 0.2759
Epoch 12

In [126]:
his.history

{'loss': [1.974126935005188,
  1.95502769947052,
  2.047053813934326,
  2.5198159217834473,
  2.4274771213531494,
  2.1925437450408936,
  2.1041531562805176,
  2.107595205307007,
  2.3730087280273438,
  2.0033090114593506,
  2.038065195083618,
  2.0677525997161865,
  1.9041016101837158,
  1.8949377536773682,
  1.859500765800476,
  1.8419746160507202,
  1.8285784721374512,
  1.815733551979065,
  1.8032647371292114,
  1.7910077571868896],
 'acc': [0.3103448152542114,
  0.3103448152542114,
  0.27586206793785095,
  0.2068965584039688,
  0.2068965584039688,
  0.27586206793785095,
  0.3103448152542114,
  0.3103448152542114,
  0.2068965584039688,
  0.27586206793785095,
  0.27586206793785095,
  0.24137930572032928,
  0.3448275923728943,
  0.3103448152542114,
  0.3448275923728943,
  0.3448275923728943,
  0.3448275923728943,
  0.37931033968925476,
  0.3448275923728943,
  0.3448275923728943]}

In [129]:
model.predict(x_test_index).argmax(axis=1)

1/1 [==============================] - 1s 1s/step


array([ 1, 18, 21,  1], dtype=int64)

In [132]:
y_test

array([10, 13, 20, 16], dtype=int64)

In [131]:
model.predict(x_train_index).argmax(axis=1)


1/1 [==============================] - 1s 519ms/step


array([ 1, 21,  2,  1,  1, 23,  1,  1, 13,  8,  1, 25,  1, 13,  1,  1,  1,
        1,  2,  1,  1, 18,  1,  8,  2, 25,  2,  3,  1], dtype=int64)

In [133]:
y_train

array([ 2, 21, 22, 15,  9, 23, 24, 19, 14,  7, 12, 25,  5, 13, 23,  2,  1,
       11,  2,  4,  1, 18, 17,  8,  6, 26,  2,  3,  0], dtype=int64)

In [130]:
x_indx_test

[array([2.670e+02, 8.100e+01, 1.020e+03, 2.180e+02, 2.680e+02, 1.510e+02,
        3.950e+02, 6.000e+00, 6.800e+01, 1.600e+01, 1.400e+01, 1.021e+03,
        1.900e+01, 2.190e+02, 2.200e+02, 5.200e+01, 5.700e+01, 1.022e+03,
        2.670e+02, 8.100e+01, 1.023e+03, 5.150e+02, 3.590e+02, 4.400e+01,
        1.660e+02, 2.000e+00, 5.160e+02, 1.800e+01, 1.024e+03, 1.000e+00,
        2.680e+02, 1.200e+01, 1.000e+00, 6.800e+01, 1.600e+01, 5.900e+01,
        3.000e+00, 1.025e+03, 1.090e+02, 2.180e+02, 3.960e+02, 1.000e+00,
        5.800e+01, 7.000e+00, 5.170e+02, 3.000e+00, 4.000e+01, 1.026e+03,
        3.800e+01, 1.027e+03, 1.028e+03, 2.670e+02, 8.100e+01, 5.200e+01,
        5.700e+01, 1.029e+03, 4.000e+00, 1.030e+03, 1.000e+00, 5.800e+01,
        3.000e+00, 1.110e+02, 2.210e+02, 2.220e+02, 3.970e+02, 2.230e+02,
        2.190e+02, 1.160e+02, 2.500e+01, 2.690e+02, 1.520e+02, 3.000e+00,
        6.900e+01, 1.520e+02, 8.000e+01, 3.980e+02, 1.160e+02, 2.500e+01,
        1.031e+03, 1.032e+03, 1.200e+0